<a href="https://colab.research.google.com/github/nourhanhamdyebrahim123/LULC_change_Alexandria/blob/main/Copy_of_LULC_change_Alex_2035.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install imagecodecs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 42.1 MB/s eta 0:00:00


In [ ]:
#!pip install tensorflow==2.13.0
import tensorflow as tf
print(tf.__version__)


2.19.0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install segmentation_models generic_utils rasterio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.6/128.6 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.3/22.3 MB 115.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 6.0 MB/s eta 0:00:00


In [ ]:
import glob
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from keras.callbacks import EarlyStopping

from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, BatchNormalization, ConvLSTM2D, Input
from keras.models import Sequential, Model, load_model
import os
import tifffile as tif
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from tensorflow.keras.utils import to_categorical
from tensorflow import keras
#import segmentation_models as sm
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import sys
from sklearn.metrics import jaccard_score, f1_score, confusion_matrix, cohen_kappa_score
from datetime import datetime

In [ ]:
data_dir = '/content/drive/My Drive/LULC_change_modeling_Alex/Dataset/'

In [ ]:
import numpy as np
import os
import glob
import tifffile as tif
import imagecodecs  # Explicit import
import rasterio  # To preserve georeferencing metadata

# Define parameters
total_frames = 288  # Adjust as needed
frames_per_sequence = 4
num_sequences = total_frames // frames_per_sequence
img_height, img_width = 256, 256
array_shape = (num_sequences, frames_per_sequence, img_height, img_width, 1)

x_test = np.zeros(array_shape, dtype=np.float32)

# To store georeference info for each sequence
geo_metadata = []

# Find all X_test image folders
paths = glob.glob(os.path.join(data_dir, '**', 'X_test', 'images'), recursive=True)

seq_idx = 0

for path in sorted(paths):
    masks = [m for m in os.listdir(path) if m.endswith('.tif')]

    # Sort by year extracted from filename (e.g., "image_1995_06.05.tif")
    masks.sort(key=lambda name: int(name.split('_')[1]))

    print(f"\nSequence folder: {path}")
    print(f"Sorted frames: {masks}")

    seq_metadata = []  # Metadata per sequence

    for frame_idx, mask_name in enumerate(masks):
        mask_path = os.path.join(path, mask_name)

        # Use rasterio to read both data and georeference
        with rasterio.open(mask_path) as src:
            mask = src.read(1)  # Read first band
            transform = src.transform
            crs = src.crs
            bounds = src.bounds

        # Handle NaNs and negative values
        mask = np.nan_to_num(mask, nan=1e-6)
        mask[mask <= 0] = 1e-6

        # Expand dims and store
        mask = np.expand_dims(mask, axis=-1)
        x_test[seq_idx, frame_idx] = mask

        # Save metadata for this frame
        seq_metadata.append({
            "filename": mask_name,
            "transform": transform,
            "crs": crs,
            "bounds": bounds
        })

        print(f"  {mask_name} → seq {seq_idx}, frame {frame_idx}")

    geo_metadata.append(seq_metadata)

    seq_idx += 1
    #if seq_idx > num_sequences:
        #break

print("\nFinal array shape_x_test:", x_test.shape)
print(f"Stored georeferenced metadata for {len(geo_metadata)} sequences.")



Sequence folder: /content/drive/My Drive/LULC_change_modeling_Alex/Dataset/Sequence_01/X_test/images
Sorted frames: ['image_1995_00.00.tif', 'image_2005_00.00.tif', 'image_2015_00.00.tif', 'image_2025_00.00.tif']
  image_1995_00.00.tif → seq 0, frame 0
  image_2005_00.00.tif → seq 0, frame 1
  image_2015_00.00.tif → seq 0, frame 2
  image_2025_00.00.tif → seq 0, frame 3

Sequence folder: /content/drive/My Drive/LULC_change_modeling_Alex/Dataset/Sequence_02/X_test/images
Sorted frames: ['image_1995_00.01.tif', 'image_2005_00.01.tif', 'image_2015_00.01.tif', 'image_2025_00.01.tif']
  image_1995_00.01.tif → seq 1, frame 0
  image_2005_00.01.tif → seq 1, frame 1
  image_2015_00.01.tif → seq 1, frame 2
  image_2025_00.01.tif → seq 1, frame 3

Sequence folder: /content/drive/My Drive/LULC_change_modeling_Alex/Dataset/Sequence_03/X_test/images
Sorted frames: ['image_1995_00.02.tif', 'image_2005_00.02.tif', 'image_2015_00.02.tif', 'image_2025_00.02.tif']
  image_1995_00.02.tif → seq 2, frame 

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, ConvLSTM2D, BatchNormalization, Dense
# import segmentation_models as sm # Removed this import as it's causing the error

# Define your custom loss and metrics again (same as in training)
# If you don't have them right now, use dummy ones temporarily.
def total_loss(y_true, y_pred):
    return tf.reduce_mean(tf.square(y_true - y_pred))  # Example: MSE

# Re-defining metrics without relying on segmentation_models
# Assuming these were the metrics used during training
def iou_score(y_true, y_pred, threshold=0.5):
    y_pred = tf.cast(y_pred > threshold, tf.float32)
    intersection = tf.reduce_sum(y_true * y_pred)
    union = tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) - intersection
    return intersection / (union + tf.keras.backend.epsilon())

def f1_score(y_true, y_pred, threshold=0.5):
    y_pred = tf.cast(y_pred > threshold, tf.float32)
    tp = tf.reduce_sum(y_true * y_pred)
    fp = tf.reduce_sum((1 - y_true) * y_pred)
    fn = tf.reduce_sum(y_true * (1 - y_pred))
    precision = tp / (tp + fp + tf.keras.backend.epsilon())
    recall = tp / (tp + fn + tf.keras.backend.epsilon())
    return 2 * (precision * recall) / (precision + recall + tf.keras.backend.epsilon())

metrics = [iou_score, f1_score]


def build_convlstm_model():
    image_input = Input(shape=(4, 256, 256, 1))

    x = ConvLSTM2D(filters=64, kernel_size=(3, 3), padding='same', return_sequences=True)(image_input)
    x = BatchNormalization()(x)
    x = ConvLSTM2D(filters=32, kernel_size=(3, 3), padding='same', return_sequences=False)(x)
    x = BatchNormalization()(x)

    # The output layer should match the number of classes in your segmentation task.
    # Based on the later use of np.argmax with axis=3, it seems you have 4 classes.
    x = Dense(4, activation='softmax')(x)

    model = Model(inputs=image_input, outputs=x)
    # Compile the model with the same optimizer and loss function used during training
    model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=metrics)
    return model

# ---- Build and load weights ----
model = build_convlstm_model()

# Load weights (not the full model, to avoid version mismatch issues)
model.load_weights("/content/drive/My Drive/LULC_change_modeling_Alex/convlstm_time_series.hdf5")

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 4, 256, 256, 1) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_lstm2d (ConvLSTM2D)        │ (None, 4, 256, 256,    │       150,016 │
│                                 │ 64)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 4, 256, 256,    │           256 │
│ (BatchNormalization)            │ 64)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_lstm2d_1 (ConvLSTM2D)      │ (None, 256, 256, 32)   │       110,720 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 256, 256, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256, 256, 4)    │           132 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 261,252 (1020.52 KB)

 Trainable params: 261,060 (1019.77 KB)

 Non-trainable params: 192 (768.00 B)

In [ ]:
predicted_image = model.predict(x_test)
predicted_image = np.argmax(predicted_image, axis=3).astype(np.uint32)


3/3 ━━━━━━━━━━━━━━━━━━━━ 15s 2s/step


In [ ]:
import os
import rasterio
import numpy as np

# === Define the output directory ===
output_directory = "/content/drive/My Drive/LULC_change_modeling_Alex/Results_2"
os.makedirs(output_directory, exist_ok=True)

# === Loop through predicted patches ===
for k in range(len(predicted_image)):
    print("shape of predicted_image[k]:", predicted_image[k].shape)

    # Get the metadata for the current sequence (k) and the last frame (index 3)
    # assuming the prediction corresponds to the last frame of the sequence
    if k < len(geo_metadata) and len(geo_metadata[k]) > 0:
        reference_metadata = geo_metadata[k][-1] # Get metadata for the last frame
        reference_path = os.path.join(data_dir, 'Sequence_{:02d}'.format(k+1), 'X_test', 'images', reference_metadata['filename'])
    else:
        print(f"Warning: Could not find metadata for sequence {k}. Skipping.")
        continue


    # --- Load georeferencing from the corresponding input patch ---
    try:
        with rasterio.open(reference_path) as src:
            meta = src.meta.copy()
            transform = src.transform
            crs = src.crs
    except Exception as e:
        print(f"Error opening reference file {reference_path}: {e}")
        continue


    # --- Update metadata for predicted output ---
    meta.update({
        "count": 1,
        "dtype": rasterio.uint32,
        "transform": transform,
        "crs": crs
    })

    # --- Construct the full output path ---
    img_name = f"prediction_{k:02}"
    output_geotiff_path = os.path.join(output_directory, img_name + ".tif")

    # --- Save predicted patch with georeferencing ---
    with rasterio.open(output_geotiff_path, "w", **meta) as dst:
        dst.write(predicted_image[k], 1)

    print(f"✅ Saved georeferenced prediction to: {output_geotiff_path}")

shape of predicted_image[k]: (256, 256)
✅ Saved georeferenced prediction to: /content/drive/My Drive/LULC_change_modeling_Alex/Results_2/prediction_00.tif
shape of predicted_image[k]: (256, 256)
✅ Saved georeferenced prediction to: /content/drive/My Drive/LULC_change_modeling_Alex/Results_2/prediction_01.tif
shape of predicted_image[k]: (256, 256)
✅ Saved georeferenced prediction to: /content/drive/My Drive/LULC_change_modeling_Alex/Results_2/prediction_02.tif
shape of predicted_image[k]: (256, 256)
✅ Saved georeferenced prediction to: /content/drive/My Drive/LULC_change_modeling_Alex/Results_2/prediction_03.tif
shape of predicted_image[k]: (256, 256)
✅ Saved georeferenced prediction to: /content/drive/My Drive/LULC_change_modeling_Alex/Results_2/prediction_04.tif
shape of predicted_image[k]: (256, 256)
✅ Saved georeferenced prediction to: /content/drive/My Drive/LULC_change_modeling_Alex/Results_2/prediction_05.tif
shape of predicted_image[k]: (256, 256)
✅ Saved georeferenced predicti

In [ ]:
from rasterio.merge import merge
import glob

# Collect all patch GeoTIFFs
# Correct the glob pattern to match the saved filenames
tif_files = sorted(glob.glob(os.path.join(output_directory, "prediction_*.tif")))

# Merge patches into one full raster
src_files_to_mosaic = [rasterio.open(fp) for fp in tif_files]
mosaic, out_transform = merge(src_files_to_mosaic)

# Update metadata
# Assuming 'meta' variable from previous cell is still available and relevant
# If not, you might need to open one of the source files to get initial metadata
if 'meta' not in locals():
    with rasterio.open(tif_files[0]) as src:
        meta = src.meta.copy()


meta.update({
    "height": mosaic.shape[1],
    "width": mosaic.shape[2],
    "transform": out_transform
})

# Save final merged GeoTIFF
merged_path = os.path.join(output_directory, "predicted_full_lulc_2035.tif")
with rasterio.open(merged_path, "w", **meta) as dst:
    dst.write(mosaic)

print("✅ Merged georeferenced full prediction saved:", merged_path)

✅ Merged georeferenced full prediction saved: /content/drive/My Drive/LULC_change_modeling_Alex/Results_2/predicted_full_lulc_2035.tif
